# Extracting data from network config files using batfish.

The code uses batfish to get vendor neutral information about the network. Batfish outputs the data in form of pandas dataframe which are then converted and stored as json files.

## Requirements:
 * Batfish tool and pybatfish.
 Both can be easily installed by following the directions on the [offical github page](https://github.com/batfish/batfish). Few things to keep in mind:
 * Pyhton version should be >=3.6
 * Java 8 is required
 * Use virtual environment to install pybatfish (as mentioned in the documentation).
 


Before running the following cells, make sure that the batfish tool is running locally. If you've followed the directions given on the github page, the following command should do it:
> docker run -v ```$(pwd)/data:/data -p 9997:9997 -p 9996:9996 batfish/allinone```

The following cell imports pybatfish and other needed packages. If you get a ```ConnectionError```, it probably means that batfish is not running locally. Other than that there's no need to understand any part of it.

In [2]:
import logging
import random
import os

import pandas as pd
from IPython.display import display
from pandas.io.formats.style import Styler

from pybatfish.client.commands import *
# noinspection PyUnresolvedReferences
from pybatfish.datamodel import Interface, Edge
from pybatfish.datamodel.flow import HeaderConstraints, PathConstraints
from pybatfish.question import bfq, load_questions  # noqa: F401
from pybatfish.util import get_html

bf_logger.setLevel(logging.WARN)

load_questions()

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
# Prevent rendering text between '$' as MathJax expressions
pd.set_option('display.html.use_mathjax', False)

# UUID for CSS styles used by pandas styler.
# Keeps our notebook HTML deterministic when displaying dataframes
_STYLE_UUID = "pybfstyle"


class MyStyler(Styler):
    """A custom styler for displaying DataFrames in HTML"""

    def __repr__(self):
        return repr(self.data)


def show(df):
    """
    Displays a dataframe as HTML table.

    Replaces newlines and double-spaces in the input with HTML markup, and
    left-aligns the text.
    """

    # workaround for Pandas bug in Python 2.7 for empty frames
    if not isinstance(df, pd.DataFrame) or df.size == 0:
        display(df)
        return
    df = df.replace('\n', '<br>', regex=True).replace('  ', '&nbsp;&nbsp;',
                                                      regex=True)
    display(MyStyler(df).set_uuid(_STYLE_UUID).format(get_html)
            .set_properties(**{'text-align': 'left', 'vertical-align': 'top'}))


## The following cell is where batfish analyzes the config files.
```NETWORK_NAME``` and ```SNAPSHOT_NAME``` don't matter if you're working with a single network. However, the final json files will be stored in a directory with name ```NETWORK_NAME json files```. The important change you should make is to ```SNAPSHOT_PATH``` which should point to the directory containing your config files. The following setup is to read the [example network](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example) given in the [pybatfish tutorials](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks).


In [3]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

# Now create the network and initialize the snapshot
bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)
load_questions()

Batfish has a property ```namedStrucures()``` which gives the named structures of the network.``` answers().frame()``` is used to get the data in a pandas dataframe. 

The column ```Structure_Type``` denotes the type of the named structure(the code will create a json file for each value in the column). 


The following block will get the named structures in dataframe ```data```.

In [4]:
data = bfq.namedStructures().answer().frame()

In [5]:
data.head()

,Node,Structure_Type,Structure_Name,Structure_Definition
0,as2core1,VRF,default,"{'name': 'default', 'bgpProcess': {'multipathEbgp': True, 'multipathEquivalentAsPathMatchMode': 'EXACT_PATH', 'multipathIbgp': True, 'neighbors': {'2.1.1.1/32': {'class': 'org.batfish.datamodel.BgpActivePeerConfig', 'additionalPathsReceive': True, 'additionalPathsSelectAll': True, 'additionalPathsSend': True, 'advertiseExternal': False, 'advertiseInactive': True, 'allowLocalAsIn': False, 'allowRemoteAsOut': True, 'clusterId': 33620481, 'defaultMetric': 0, 'ebgpMultihop': False, 'enforceFirstAs': False, 'exportPolicy': '~BGP_PEER_EXPORT_POLICY:default:2.1.1.1~', 'group': 'as2', 'localAs': 2, 'localIp': '2.1.2.1', 'peerAddress': '2.1.1.1', 'remoteAs': 2, 'routeReflectorClient': True, 'sendCommunity': True}, '2.1.1.2/32': {'class': 'org.batfish.datamodel.BgpActivePeerConfig', 'additionalPathsReceive': True, 'additionalPathsSelectAll': True, 'additionalPathsSend': True, 'advertiseExternal': False, 'advertiseInactive': True, 'allowLocalAsIn': False, 'allowRemoteAsOut': True, 'clusterId': 33620481, 'defaultMetric': 0, 'ebgpMultihop': False, 'enforceFirstAs': False, 'exportPolicy': '~BGP_PEER_EXPORT_POLICY:default:2.1.1.2~', 'group': 'as2', 'localAs': 2, 'localIp': '2.1.2.1', 'peerAddress': '2.1.1.2', 'remoteAs': 2, 'routeReflectorClient': True, 'sendCommunity': True}, '2.1.3.1/32': {'class': 'org.batfish.datamodel.BgpActivePeerConfig', 'additionalPathsReceive': True, 'additionalPathsSelectAll': True, 'additionalPathsSend': True, 'advertiseExternal': False, 'advertiseInactive': True, 'allowLocalAsIn': False, 'allowRemoteAsOut': True, 'clusterId': 33620481, 'defaultMetric': 0, 'ebgpMultihop': False, 'enforceFirstAs': False, 'exportPolicy': '~BGP_PEER_EXPORT_POLICY:default:2.1.3.1~', 'group': 'as2', 'localAs': 2, 'localIp': '2.1.2.1', 'peerAddress': '2.1.3.1', 'remoteAs': 2, 'routeReflectorClient': True, 'sendCommunity': True}, '2.1.3.2/32': {'class': 'org.batfish.datamodel.BgpActivePeerConfig', 'additionalPathsReceive': True, 'additionalPathsSelectAll': True, 'additionalPathsSend': True, 'advertiseExternal': False, 'advertiseInactive': True, 'allowLocalAsIn': False, 'allowRemoteAsOut': True, 'clusterId': 33620481, 'defaultMetric': 0, 'ebgpMultihop': False, 'enforceFirstAs': False, 'exportPolicy': '~BGP_PEER_EXPORT_POLICY:default:2.1.3.2~', 'group': 'as2', 'localAs': 2, 'localIp': '2.1.2.1', 'peerAddress': '2.1.3.2', 'remoteAs': 2, 'routeReflectorClient': True, 'sendCommunity': True}}, 'routerId': '2.1.2.1', 'tieBreaker': 'ARRIVAL_ORDER'}, 'interfaces': ['Ethernet0/0', 'GigabitEthernet0/0', 'GigabitEthernet1/0', 'GigabitEthernet2/0', 'GigabitEthernet3/0', 'Loopback0'], 'ospfProcess': {'areas': {'1': {'injectDefaultRoute': True, 'interfaces': ['GigabitEthernet0/0', 'GigabitEthernet1/0', 'GigabitEthernet2/0', 'GigabitEthernet3/0', 'Loopback0'], 'metricOfDefaultRoute': 0, 'name': 1, 'stubType': 'NONE'}}, 'exportPolicy': '~OSPF_EXPORT_POLICY:default~', 'processId': '1', 'referenceBandwidth': 100000000.0, 'routerId': '2.1.2.1'}}"
1,as3border2,IP_Access_List,103,"{'name': '103', 'lines': [{'action': 'PERMIT', 'matchCondition': {'class': 'org.batfish.datamodel.acl.MatchHeaderSpace', 'headerSpace': {'dstIps': {'class': 'org.batfish.datamodel.IpWildcardIpSpace', 'ipWildcard': '255.255.255.0'}, 'negate': False, 'srcIps': {'class': 'org.batfish.datamodel.IpWildcardIpSpace', 'ipWildcard': '3.0.1.0'}}}, 'name': 'permit ip host 3.0.1.0 host 255.255.255.0'}, {'action': 'PERMIT', 'matchCondition': {'class': 'org.batfish.datamodel.acl.MatchHeaderSpace', 'headerSpace': {'dstIps': {'class': 'org.batfish.datamodel.IpWildcardIpSpace', 'ipWildcard': '255.255.255.0'}, 'negate': False, 'srcIps': {'class': 'org.batfish.datamodel.IpWildcardIpSpace', 'ipWildcard': '3.0.2.0'}}}, 'name': 'permit ip host 3.0.2.0 host 255.255.255.0'}], 'sourceName': '103', 'sourceType': 'extended ipv4 access-list'}"
2,as1border1,IP_Access_List,103,"{'name': '103', 'lines': [{'action': 'PERMIT', 'matchCo

The following block will get all the different values for ```Structure_Type``` and will split the data set on each value. Each named structure is then saved in the following format: ```Structure_type.json```. For example, named structures with ```Structure_Type == "Routing_Policy" ``` will be saved as ```Routing_policy.json```. The files will be stored in a directory ```NETWORK_NAME_json_files```(directory will be created if it doesn't already exists).

In [5]:
Structure_types = list(data.Structure_Type.unique())
for struct in Structure_types:
    df = data[data['Structure_Type']==struct]
    del df['Structure_Type']
    fileName=str(struct)+".json"
    directory = "./"+str(NETWORK_NAME)+" json files"
    if not os.path.exists(directory):
        os.mkdir(directory)
    fullName = os.path.join(directory, fileName)
    df.to_json(fullName,index='False',orient="split") 

print("JSON files saved")

JSON files saved


Apart from named structures we also need the node properties. Batfish has a property called ```nodeProperties()```  which can be used. The node properties will be saved as ```nodeProperties.json```. It will be saved in the same directory as the named structures.

## TODO: extract only required properties from node structure for saving.

In [6]:
nodeProperties = bfq.nodeProperties().answer().frame()
nodeProperties.head()

,Node,AS_Path_Access_Lists,Authentication_Key_Chains,Canonical_IP,Community_Lists,Configuration_Format,Default_Cross_Zone_Action,Default_Inbound_Action,Device_Type,DNS_Servers,DNS_Source_Interface,Domain_Name,Hostname,IKE_Gateways,IKE_Policies,Interfaces,IP_Access_Lists,IP_Spaces,IP6_Access_Lists,IPSec_Policies,IPSec_Proposals,IPSec_Vpns,Logging_Servers,Logging_Source_Interface,NTP_Servers,NTP_Source_Interface,Route_Filter_Lists,Route6_Filter_Lists,Routing_Policies,SNMP_Source_Interface,SNMP_Trap_Servers,TACACS_Servers,TACACS_Source_Interface,Vendor_Family,VRFs,Zones
0,as1border1,[],[],1.0.1.1,"['as3_community', 'as1_community', 'as2_community']",CISCO_IOS,PERMIT,PERMIT,ROUTER,[],None,lab.local,as1border1,[],[],"['GigabitEthernet0/0', 'GigabitEthernet1/0', 'Ethernet0/0', 'Loopback0']","['101', '102', '103']",[],[],[],[],[],[],None,[],None,"['101', '102', '103', 'default_list', 'inbound_route_filter']",[],"['as3_to_as1', '~BGP_PEER_EXPORT_POLICY:default:5.6.7.8~', '~OSPF_EXPORT_POLICY:default~', '~BGP_PEER_EXPORT_POLICY:default:3.2.2.2~', '~BGP_PEER_EXPORT_POLICY:default:1.10.1.1~', 'as1_to_as2', 'as1_to_as3', '~BGP_PEER_EXPORT_POLICY:default:10.12.11.2~', 'as2_to_as1', '~BGP_COMMON_EXPORT_POLICY:default~']",None,[],[],None,CISCO,['default'],[]
1,as3border1,[],[],3.0.1.1,"['as3_community', 'as1_community', 'as2_community']",CISCO_IOS,PERMIT,PERMIT,ROUTER,[],None,lab.local,as3border1,[],[],"['GigabitEthernet0/0', 'GigabitEthernet1/0', 'Ethernet0/0', 'Loopback0']","['101', '102', '103']",[],[],[],[],[],[],None,"['18.18.18.18', '23.23.23.23']",None,"['101', '102', '103', 'default_list', 'inbound_route_filter']",[],"['as3_to_as1', 'as3_to_as2', '~OSPF_EXPORT_POLICY:default~', '~BGP_PEER_EXPORT_POLICY:default:3.10.1.1~', '~BGP_PEER_EXPORT_POLICY:default:10.23.21.2~', 'as1_to_as3', 'as2_to_as3', '~BGP_COMMON_EXPORT_POLICY:default~']",None,[],[],None,CISCO,['default'],[]
2,host2,[],[],2.128.1.101,[],HOST,PERMIT,PERMIT,HOST,[],None,None,host2,[],[],['eth0'],"['filter::FORWARD', 'nat::POSTROUTING', 'nat::PREROUTING', 'filter::INPUT', 'mangle::POSTROUTING', 'mangle::INPUT', 'filter::OUTPUT', 'mangle::FORWARD', 'mangle::OUTPUT', 'mangle::PREROUTING', 'nat::OUTPUT']",[],[],[],[],[],[],None,[],None,[],[],[],None,[],[],None,,['default'],[]
3,as2core2,[],[],2.1.2.2,[],CISCO_IOS,PERMIT,PERMIT,ROUTER,[],None,lab.local,as2core2,[],[],"['GigabitEthernet0/0', 'GigabitEthernet1/0', 'GigabitEthernet2/0', 'GigabitEthernet3/0', 'Ethernet0/0', 'Loopback0']",[],[],[],[],[],[],"['1.1.1.1', '2.2.2.2']",None,[],None,[],[],"['~BGP_PEER_EXPORT_POLICY:default:2.1.3.2~', '~BGP_PEER_EXPORT_POLICY:default:2.1.3.1~', '~OSPF_EXPORT_POLICY:default~', '~BGP_PEER_EXPORT_POLICY:default:2.1.1.2~', '~BGP_PEER_EXPORT_POLICY:default:2.1.1.1~', '~BGP_COMMON_EXPORT_POLICY:default~']",None,[],[],None,CISCO,['default'],[]
4,as3border2,[],[],3.0.2.1,"['as3_community', 'as1_community', 'as2_community']",CISCO_IOS,PERMIT,PERMIT,ROUTER,[],None,lab.local,as3border2,[],[],"['GigabitEthernet0/0', 'GigabitEthernet1/0', 'Ethernet0/0', 'Loopback0']","['101', '102', '103']",[],[],[],[],[],[],None,"['18.18.18.18', '23.23.23.23']",None,"['101', '102', '103', 'inbound_route_filter']",[],"['as3_to_as1', 'as3_to_as2', '~OSPF_EXPORT_POLICY:default~', '~BGP_PEER_EXPORT_POLICY:default:3.10.1.1~', '~BGP_PEER_EXPORT_POLICY:default:10.13.22.1~', 'as1_to_as3', 'as2_to_as3', '~BGP_COMMON_EXPORT_POLICY:default~']",None,[],[],None,CISCO,['default'],[]


In [7]:
nodeProperties.to_json(os.path.join(directory, "nodeProperties.json"),orient="records",lines="True")